# Extract Features

1. Read csv files and load as dfs
2. Combine dfs
3. Get POS, NER
4. Save in new features_df
5. Get semantic cosine similarity

In [1]:
import os, sys

import pandas as pd

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from log_files import LogData
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
df = log_files.read_data(notebook_dir, log_file_path, predictions)
df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/prediction_logs/batch_1-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/prediction_logs/batch_1-predictions/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/prediction_logs/batch_2-predictions
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/prediction_logs/batch_2-predictions/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.",1,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.",1,finance,gpt-4o,NAVI_GATOR,0,2
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1
3,"On August 21, 2023, Goldman Sachs speculates that the stock price at Amazon will likely increase.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,2
4,"George, a financial analyst, predicts that on 01/15/2024, the Google revenue may rise.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,3
5,"According to BlackRock, the operating cash flow at ExxonMobil would fall in Q3 2023.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,4
6,"In Q2 2023, Wells Fargo envisions that the stock price at Tesla has some probability to remain stable.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,5


In [4]:
df[66 : 67]

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
66,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1


In [5]:
df[2 : 3]

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,finance,mixtral-8x7b-instruct,NAVI_GATOR,0,1


- issue: `obesity rate` is not being captures by POS or NER.

In [6]:
only_predictions = DataProcessing.df_to_list(df, 'Base Sentence')
print(only_predictions)

sfe_class = SpacyFeatureExtraction(df, "Base Sentence")
word_leveL_disable_components = ["lemmatizer"]
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings =  sfe_class.extract_features(data=only_predictions, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
pred_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
pred_pos_ner_features_df = DataProcessing.concat_dfs(pred_pos_ner_features, axis=1, ignore_index=False)
pred_pos_ner_features_df.head()

['On August 21, 2024, as a financial analyst, I predict that the revenue at Amazon will potentially decrease in Q3 of 2027.', 'On 2024-08-21, Morgan Stanley speculates the operating income at Amazon will likely increase.', 'According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.', 'On August 21, 2023, Goldman Sachs speculates that the stock price at Amazon will likely increase.', 'George, a financial analyst, predicts that on 01/15/2024, the Google revenue may rise.', 'According to BlackRock, the operating cash flow at ExxonMobil would fall in Q3 2023.', 'In Q2 2023, Wells Fargo envisions that the stock price at Tesla has some probability to remain stable.', 'The stock price at Apple should stay the same in August 2023, according to Morgan Stanley.', 'Dr. Smith predicts that the prevalence of chronic illnesses at rural health clinics will potentially decrease in Q4 of 2026.', 'WHO forecasts that the prevalence of chronic illnesses at global urban

,ADP_4,PUNCT_3,PUNCT_5,PROPN_7,DET_2,NOUN_2,VERB_4,NOUN_7,NOUN_8,NOUN_3,VERB_3,ADV_1,VERB_2,NUM_3,ADP_1,DET_3,NUM_1,ADP_3,PROPN_1,SYM_2,PART_2,ADJ_2,PUNCT_2,ADJ_1,ADP_5,PROPN_5,PART_1,PROPN_2,VERB_5,PRON_1,NOUN_6,PUNCT_6,AUX_2,DET_4,ADP_2,PROPN_6,AUX_1,NUM_2,PROPN_3,ADJ_3,DET_1,ADJ_4,PUNCT_1,NOUN_5,SYM_1,PUNCT_4,VERB_1,SCONJ_1,NOUN_4,NOUN_1,CCONJ_1,PROPN_4,DET_5,NORP_1,ORG_1,EVENT_1,DATE_1,CARDINAL_1,GPE_1,TIME_1,ORG_2,GPE_2,DATE_2,ORG_3,PERCENT_1,LOC_1,PERSON_1
0,in,",",NaN,NaN,the,revenue,NaN,NaN,NaN,NaN,NaN,potentially,decrease,2027,On,NaN,21,at,August,NaN,NaN,NaN,",",financial,of,NaN,NaN,Amazon,NaN,I,NaN,NaN,NaN,NaN,as,NaN,will,2024,Q3,NaN,a,NaN,",",NaN,NaN,.,predict,that,NaN,analyst,NaN,NaN,NaN,NaN,Amazon,NaN,"August 21, 2024",NaN,Q3,NaN,NaN,NaN,2027,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,income,NaN,NaN,NaN,NaN,NaN,likely,increase,21,On,NaN,2024,NaN,Morgan,-,NaN,NaN,.,NaN,NaN,NaN,NaN,Stanley,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,will,08,Amazon,NaN,the,NaN,",",NaN,-,NaN,speculates,NaN,NaN,operating,NaN,NaN,NaN,NaN,Morgan Stanley,NaN,2024-08-21,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,NaN
2,of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decrease,NaN,expected,NaN,to,NaN,2024,in,JPMorgan,NaN,NaN,NaN,.,net,NaN,NaN,to,Chase,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,is,NaN,Microsoft,NaN,the,NaN,",",NaN,NaN,NaN,According,NaN,NaN,profit,NaN,Q2,NaN,NaN,JPMorgan Chase,NaN,2024,NaN,NaN,NaN,Microsoft,NaN,NaN,Q2,NaN,NaN,NaN
3,NaN,.,NaN,NaN,NaN,price,NaN,NaN,NaN,NaN,NaN,likely,increase,NaN,On,NaN,21,NaN,August,NaN,NaN,NaN,",",NaN,NaN,NaN,NaN,Goldman,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,will,2023,Sachs,NaN,the,NaN,",",NaN,NaN,NaN,speculates,that,NaN,stock,NaN,Amazon,NaN,NaN,Goldman Sachs,NaN,"August 21, 2023",NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,",",NaN,NaN,the,revenue,NaN,NaN,NaN,NaN,NaN,NaN,rise,NaN,on,NaN,01/15/2024,NaN,George,NaN,NaN,NaN,",",financial,NaN,NaN,NaN,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,may,NaN,NaN,NaN,a,NaN,",",NaN,NaN,.,predicts,that,NaN,analyst,NaN,NaN,NaN,NaN,Google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,George


## Observations

In [7]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/observation_logs/batch_1-observations
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/observation_logs/batch_1-observations/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/observation_logs/batch_2-observations
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/prediction_correctness/../data/observation_logs/batch_2-observations/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,"The financial reporter, Sarah, witnessed the stock price at Tesla plummeted in Q2 of 2023.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,1
1,"From May 15, 2025, the financial analyst speculated the net profit at Amazon was drastically different.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,2
2,"On July 1, 2024, the financial expert heard that the revenue at Google remained stable.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,5
3,"Apple's operating income generally rose in August 21, 2027, according to the financial top executive.",0,finance,gpt-3.5-turbo,NAVI_GATOR,0,6
4,Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0,finance,gpt-4o,NAVI_GATOR,0,1
5,"From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0,finance,gpt-4o,NAVI_GATOR,0,2
6,"2026/12/01, a financial analyst heard that the operating income at Amazon remained stable.",0,finance,gpt-4o,NAVI_GATOR,0,5


In [8]:
only_observations = DataProcessing.df_to_list(observations_df, 'Base Sentence')
# print(only_observations)
word_level_pos_tags, word_level_pos_mappings, word_level_ner_entities, word_level_ner_mappings = sfe_class.extract_features(data=only_observations, disable_components=word_leveL_disable_components)
all_word_level_pos_df = DataProcessing.convert_to_df(word_level_pos_tags, mapping=word_level_pos_mappings)
all_word_level_ner_df = DataProcessing.convert_to_df(word_level_ner_entities, word_level_ner_mappings)
obs_pos_ner_features = [all_word_level_pos_df, all_word_level_ner_df]
obs_pos_ner_features_df = DataProcessing.concat_dfs(obs_pos_ner_features, axis=1, ignore_index=False)
obs_pos_ner_features_df.head(3)

Spacy Doc (0):  The financial reporter, Sarah, witnessed the stock price at Tesla plummeted in Q2 of 2023.
Spacy Doc (1):  From May 15, 2025, the financial analyst speculated the net profit at Amazon was drastically different.
Spacy Doc (2):  On July 1, 2024, the financial expert heard that the revenue at Google remained stable.
Spacy Doc (3):  Apple's operating income generally rose in August 21, 2027, according to the financial top executive.


,ADP_4,PUNCT_3,PROPN_7,PUNCT_5,DET_2,NOUN_2,VERB_4,NOUN_7,NOUN_8,NOUN_3,VERB_3,ADV_1,VERB_2,NUM_3,ADP_1,DET_3,NUM_1,ADP_3,PROPN_1,SYM_2,ADJ_2,PUNCT_2,ADJ_1,ADP_6,ADP_5,PROPN_5,PART_1,PROPN_2,PRON_1,NOUN_6,ADJ_5,AUX_2,DET_4,ADP_2,PROPN_6,AUX_1,NUM_2,PROPN_3,ADJ_3,DET_1,ADJ_4,SCONJ_2,PUNCT_1,NOUN_5,SYM_1,PUNCT_4,VERB_1,SCONJ_1,NOUN_4,NOUN_1,ADP_8,CCONJ_1,PROPN_4,DET_5,ADJ_6,ADP_7,FAC_1,NORP_1,ORG_1,EVENT_1,DATE_1,GPE_1,TIME_1,ORG_2,DATE_2,CARDINAL_1,LOC_1,PERSON_1
0,NaN,.,NaN,NaN,the,stock,NaN,NaN,NaN,price,NaN,NaN,plummeted,NaN,at,NaN,2023,of,Sarah,NaN,NaN,",",financial,NaN,NaN,NaN,NaN,Tesla,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,NaN,Q2,NaN,The,NaN,NaN,",",NaN,NaN,NaN,witnessed,NaN,NaN,reporter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tesla,NaN,NaN,NaN,NaN,Q2,NaN,2023,NaN,Sarah
1,NaN,.,NaN,NaN,the,profit,NaN,NaN,NaN,NaN,NaN,drastically,NaN,NaN,From,NaN,15,NaN,May,NaN,net,",",financial,NaN,NaN,NaN,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,at,NaN,was,2025,NaN,different,the,NaN,NaN,",",NaN,NaN,NaN,speculated,NaN,NaN,analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Amazon,NaN,"May 15, 2025",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,.,NaN,NaN,the,revenue,NaN,NaN,NaN,NaN,NaN,NaN,remained,NaN,On,NaN,1,NaN,July,NaN,stable,",",financial,NaN,NaN,NaN,NaN,Google,NaN,NaN,NaN,NaN,NaN,at,NaN,NaN,2024,NaN,NaN,the,NaN,NaN,",",NaN,NaN,NaN,heard,that,NaN,expert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google,NaN,"July 1, 2024",NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Both

In [9]:
pred_sentence_features_df = DataProcessing.include_sentence_and_label(pred_pos_ner_features_df, df)
pred_sentence_features_df.head(3)
pred_profit_filt = (pred_pos_ner_features_df.NOUN_1.values == 'profit')
pred_profit_df = pred_pos_ner_features_df.loc[pred_profit_filt == True]
pred_profit_df

,Base Sentence,Sentence Label,ADP_4,PUNCT_3,PUNCT_5,PROPN_7,DET_2,NOUN_2,VERB_4,NOUN_7,NOUN_8,NOUN_3,VERB_3,ADV_1,VERB_2,NUM_3,ADP_1,DET_3,NUM_1,ADP_3,PROPN_1,SYM_2,PART_2,ADJ_2,PUNCT_2,ADJ_1,ADP_5,PROPN_5,PART_1,PROPN_2,VERB_5,PRON_1,NOUN_6,PUNCT_6,AUX_2,DET_4,ADP_2,PROPN_6,AUX_1,NUM_2,PROPN_3,ADJ_3,DET_1,ADJ_4,PUNCT_1,NOUN_5,SYM_1,PUNCT_4,VERB_1,SCONJ_1,NOUN_4,NOUN_1,CCONJ_1,PROPN_4,DET_5,NORP_1,ORG_1,EVENT_1,DATE_1,CARDINAL_1,GPE_1,TIME_1,ORG_2,GPE_2,DATE_2,ORG_3,PERCENT_1,LOC_1,PERSON_1
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decrease,NaN,expected,NaN,to,NaN,2024,in,JPMorgan,NaN,NaN,NaN,.,net,NaN,NaN,to,Chase,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,is,NaN,Microsoft,NaN,the,NaN,",",NaN,NaN,NaN,According,NaN,NaN,profit,NaN,Q2,NaN,NaN,JPMorgan Chase,NaN,2024,NaN,NaN,NaN,Microsoft,NaN,NaN,Q2,NaN,NaN,NaN
34,"According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.",1,of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fall,NaN,to,NaN,2027,in,JPMorgan,NaN,NaN,NaN,.,net,NaN,Q3,NaN,Chase,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,would,NaN,Goldman,NaN,the,NaN,",",NaN,NaN,NaN,According,NaN,NaN,profit,NaN,Sachs,NaN,NaN,JPMorgan Chase,NaN,2027,NaN,Q3,NaN,Goldman Sachs,NaN,NaN,NaN,NaN,NaN,NaN
65,Goldman Sachs forecasts that the net profit at Amazon potentially decrease in Q4 of 2025.,1,NaN,NaN,NaN,NaN,NaN,decrease,NaN,NaN,NaN,NaN,NaN,potentially,NaN,NaN,at,NaN,2025,of,Goldman,NaN,NaN,NaN,NaN,net,NaN,NaN,NaN,Sachs,NaN,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,NaN,Amazon,NaN,the,NaN,.,NaN,NaN,NaN,forecasts,that,NaN,profit,NaN,Q4,NaN,NaN,Goldman Sachs,NaN,2025,NaN,Q4,NaN,Amazon,NaN,NaN,NaN,NaN,NaN,NaN
66,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",1,of,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,decrease,NaN,expected,NaN,to,NaN,2024,in,JPMorgan,NaN,NaN,NaN,.,net,NaN,NaN,to,Chase,NaN,NaN,NaN,NaN,NaN,NaN,at,NaN,is,NaN,Microsoft,NaN,the,NaN,",",NaN,NaN,NaN,According,NaN,NaN,profit,NaN,Q2,NaN,NaN,JPMorgan Chase,NaN,2024,NaN,NaN,NaN,Microsoft,NaN,NaN,Q2,NaN,NaN,NaN


In [10]:
pred_profit_sentences = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_2 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_3 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')
# pred_profit_sentences_4 = DataProcessing.df_to_list(pred_profit_df, 'Base Sentence')


In [11]:
obs_profit_sentence_df = DataProcessing.include_sentence_and_label(obs_pos_ner_features_df, observations_df)
obs_profit_filt = (obs_pos_ner_features_df.NOUN_1.values == 'profit')
obs_profit_df = obs_pos_ner_features_df.loc[obs_profit_filt == True]
obs_profit_df

,Base Sentence,Sentence Label,ADP_4,PUNCT_3,PROPN_7,PUNCT_5,DET_2,NOUN_2,VERB_4,NOUN_7,NOUN_8,NOUN_3,VERB_3,ADV_1,VERB_2,NUM_3,ADP_1,DET_3,NUM_1,ADP_3,PROPN_1,SYM_2,ADJ_2,PUNCT_2,ADJ_1,ADP_6,ADP_5,PROPN_5,PART_1,PROPN_2,PRON_1,NOUN_6,ADJ_5,AUX_2,DET_4,ADP_2,PROPN_6,AUX_1,NUM_2,PROPN_3,ADJ_3,DET_1,ADJ_4,SCONJ_2,PUNCT_1,NOUN_5,SYM_1,PUNCT_4,VERB_1,SCONJ_1,NOUN_4,NOUN_1,ADP_8,CCONJ_1,PROPN_4,DET_5,ADJ_6,ADP_7,FAC_1,NORP_1,ORG_1,EVENT_1,DATE_1,GPE_1,TIME_1,ORG_2,DATE_2,CARDINAL_1,LOC_1,PERSON_1
4,Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,plummeted,15,at,NaN,2023,NaN,Goldman,-,NaN,NaN,net,NaN,NaN,NaN,NaN,Sachs,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,11,Tesla,NaN,the,NaN,NaN,.,NaN,-,NaN,saw,NaN,NaN,profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goldman Sachs,NaN,2023-11-15,NaN,NaN,Tesla,NaN,NaN,NaN,NaN
5,"From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,drastically,NaN,30,From,NaN,2025,NaN,Morgan,-,different,.,gross,NaN,NaN,NaN,NaN,Stanley,NaN,NaN,NaN,NaN,NaN,at,NaN,was,03,Apple,NaN,the,NaN,NaN,",",NaN,-,NaN,speculated,NaN,NaN,profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morgan Stanley,NaN,2025-03-30,NaN,NaN,Apple,NaN,NaN,NaN,NaN
49,Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,plummeted,15,at,NaN,2023,NaN,Goldman,-,NaN,NaN,net,NaN,NaN,NaN,NaN,Sachs,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,11,JPMorgan,NaN,the,NaN,NaN,.,NaN,-,NaN,saw,NaN,NaN,profit,NaN,NaN,Chase,NaN,NaN,NaN,NaN,NaN,Goldman Sachs,NaN,2023-11-15,NaN,NaN,JPMorgan Chase,NaN,NaN,NaN,NaN
94,Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,plummeted,15,at,NaN,2023,NaN,Goldman,-,NaN,NaN,net,NaN,NaN,NaN,NaN,Sachs,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,11,Apple,NaN,the,NaN,NaN,.,NaN,-,NaN,saw,NaN,NaN,profit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Goldman Sachs,NaN,2023-11-15,NaN,NaN,Apple,NaN,NaN,NaN,NaN


In [12]:
obs_profit_sentences = DataProcessing.df_to_list(obs_profit_df, 'Base Sentence')
obs_profit_sentences

['Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.',
 'From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.',
 'Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.',
 'Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.']

In [13]:
pred_obs_dict = {}
for pred_sent_idx in range(len(pred_profit_sentences)):
    pred_sentence = pred_profit_sentences[pred_sent_idx]
    pred_obs_dict[pred_sentence] = obs_profit_sentences
pred_obs_dict

{'According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.': ['Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.',
  'From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.',
  'Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.',
  'Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.'],
 'According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.': ['Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.',
  'From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.',
  'Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.',
  'Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.'],
 'Goldman Sachs forecasts that the net profit at Amazon potentially decrease in Q4 of 2025.': ['Goldman Sachs saw the net profit at Tesla plummeted in 2023-1

In [14]:
import spacy

load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in pred_obs_dict.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in pred_obs_dict[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings

# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df['Scores'] = "Similarity"
df

Prediction: According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.
    Observation: Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.
    Similarity: 0.625748872756958

    Observation: From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.
    Similarity: 0.6004109978675842

    Observation: Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.
    Similarity: 0.6323454976081848

    Observation: Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.
    Similarity: 0.6263935565948486

Prediction: According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.
    Observation: Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.
    Similarity: 0.6256189346313477

    Observation: From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.
    Similarity: 0.596201479434967

    Obse

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_51301/1625276997.py:20: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


,Prediction,Observation,Spacy Similarity,Scores
0,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0.625749,Similarity
1,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.","From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0.600411,Similarity
2,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.,0.632345,Similarity
3,"According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024.",Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.,0.626394,Similarity
4,"According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.",Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0.625619,Similarity
5,"According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.","From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0.596201,Similarity
6,"According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.",Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.,0.653105,Similarity
7,"According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.",Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.,0.617825,Similarity
8,Goldman Sachs forecasts that the net profit at Amazon potentially decrease in Q4 of 2025.,Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.,0.534696,Similarity
9,Goldman Sachs forecasts that the net profit at Amazon potentially decrease in Q4 of 2025.,"From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.",0.476514,Similarity


In [15]:
table = pd.pivot_table(df, values='Spacy Similarity', index=['Prediction', 'Observation'],
                       columns=['Scores'])

In [16]:
table

Scores                                                                                                                                                                                           Similarity
Prediction                                                                                      Observation                                                                                                
According to JPMorgan Chase, the net profit at Goldman Sachs would fall in Q3 of 2027.          From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.    0.596201
                                                                                                Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.                                 0.617825
                                                                                                Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.                        0.653105
                                                                                                Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.                                 0.625619
According to JPMorgan Chase, the net profit at Microsoft is expected to decrease in Q2 of 2024. From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.    0.600411
                                                                                                Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.                                 0.626394
                                                                                                Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.                        0.632345
                                                                                                Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.                                 0.625749
Goldman Sachs forecasts that the net profit at Amazon potentially decrease in Q4 of 2025.       From 2025-03-30, Morgan Stanley speculated the gross profit at Apple was drastically different.    0.476514
                                                                                                Goldman Sachs saw the net profit at Apple plummeted in 2023-11-15.                                 0.531176
                                                                                                Goldman Sachs saw the net profit at JPMorgan Chase plummeted in 2023-11-15.                        0.549114
                                                                                                Goldman Sachs saw the net profit at Tesla plummeted in 2023-11-15.                                 0.534696

In [ ]:
Methods to alter Precision and Recall for prob rather than binary
    
    https://en.wikipedia.org/wiki/Fuzzy_classification

Support or deny classes